In [15]:
# import everything we may need
import mysql.connector
from mysql.connector import errorcode
import config
import requests
import time
import json

In [16]:
#get data from box office mojo as json file
f = open('mojo_data.json')
data = json.load(f)

In [45]:
# connect to AWS
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)

# Establish cursor
cursor = cnx.cursor()

In [ ]:
## function to close the connection
def close_connections():
    cursor.close()
    conn.close()

In [ ]:
#name the db
db_name = 'Mod_1_Project'

In [ ]:
# create database in AWS
def create_database(cursor, database_name):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database_name))
    except mysql.connector.Error as err:
        ## Catch the error if an error occurs.
        print("Failed creating database: {}".format(err))
        exit(1)
        
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [19]:
#get list of movies from box office mojo data
titles = [x['title'] for x in data]
titles

['The Boss Baby',
 "A Dog's Purpose",
 'Jigsaw',
 'Guardians of the Galaxy Vol. 2',
 'Split',
 'Jumanji:Welcome to the Jungle',
 'All Eyez on Me',
 'Downsizing',
 'Annabelle:Creation',
 'Despicable Me 3',
 'Spider-Man:Homecoming',
 "Molly's Game",
 'The LEGO Batman Movie',
 'American Assassin',
 'Geostorm',
 'Star Wars:The Last Jedi',
 'Transformers:The Last Knight',
 'Baby Driver',
 'Ghost in the Shell (2017)',
 'Atomic Blonde',
 'Beauty and the Beast (2017)',
 'Justice League',
 'Logan Lucky',
 "Tyler Perry's Boo 2! A Madea Halloween",
 'Rings',
 'Victoria and Abdul',
 'Smurfs:The Lost Village',
 'I, Tonya',
 'Baywatch',
 'The Shape of Water',
 'Rough Night',
 'Thor:Ragnarok',
 'Valerian and the City of a Thousand Planets',
 'Alien:Covenant',
 'American Made',
 'Captain Underpants:The First Epic Movie',
 'The Dark Tower',
 'Get Out',
 'Lady Bird',
 'Coco',
 'The Post',
 'John Wick:Chapter Two',
 'The Foreigner',
 'The Fate of the Furious',
 'Leap!',
 'Kidnap (2017)',
 'Pirates of the

In [27]:
key = config.apiKey
url = 'http://www.omdbapi.com/?i=tt3896198&apikey=6bc5a4e2'

# Make call to API
# Iterate over list of titles (from box office mojo) to retrieve all

def OMDB_call(titles):
    all_titles = []
    for title in titles: 
        parameters = {'t': {title}}
        response = requests.get(url, params=parameters)
        data = response.json()
        all_titles.append(data)
        time.sleep(.5)
    return all_titles

movie_dict = OMDB_call(titles[0:50])

In [50]:
movie_dict

[{'Title': 'The Boss Baby',
  'Year': '2017',
  'Rated': 'PG',
  'Released': '31 Mar 2017',
  'Runtime': '97 min',
  'Genre': 'Animation, Adventure, Comedy, Family, Fantasy',
  'Director': 'Tom McGrath',
  'Writer': 'Michael McCullers, Marla Frazee (based on the book by)',
  'Actors': 'Alec Baldwin, Steve Buscemi, Jimmy Kimmel, Lisa Kudrow',
  'Plot': 'A suit-wearing, briefcase-carrying baby pairs up with his 7-year old brother to stop the dastardly plot of the CEO of Puppy Co.',
  'Language': 'English, Spanish',
  'Country': 'USA',
  'Awards': 'Nominated for 1 Oscar. Another 1 win & 19 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMTg5MzUxNzgxNV5BMl5BanBnXkFtZTgwMTM2NzQ3MjI@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.3/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '52%'},
   {'Source': 'Metacritic', 'Value': '50/100'}],
  'Metascore': '50',
  'imdbRating': '6.3',
  'imdbVotes': '94,906',
  'imdbID': 'tt3874544',
  'Type': 'm

In [35]:
#function to create tables in our database
def create_table(dict_of_tables, db_name):
    for table_name in dict_of_tables:
        table_query = dict_of_tables[table_name]
        cursor.execute("USE {}".format(db_name))
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(table_query)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")


In [46]:
#Create tables to input data
TABLES = {}
TABLES['main_movie_table'] = (
    "CREATE TABLE main_movie_table ("
    "  movie_id int NOT NULL AUTO_INCREMENT,"
    "  title text NOT NULL,"
    "  budget int(10),"
    "  release_date date,"
    "  award_wins int(3),"
    "  award_nomination int(3),"
    "  RT_rating int(2),"
    "  IMDB_rating real,"
    "  PRIMARY KEY (movie_id)"
    ") ENGINE=InnoDB")

TABLES['genres'] = (
    "CREATE TABLE genres ("
    "  genre_id int NOT NULL,"
    "  genre text,"
    "  PRIMARY KEY (genre_id)"
    ") ENGINE=InnoDB")

TABLES['key_words'] = (
    "CREATE TABLE key_words ("
    "  keyword_id int NOT NULL,"
    "  key_words text,"
    "  PRIMARY KEY (keyword_id)"
    ") ENGINE=InnoDB")

TABLES['movies_genres'] = (
    "CREATE TABLE movies_genres ("
      " movie_id int NOT NULL,"
      " genre_id int NOT NULL"
      ") ENGINE=InnoDB")

TABLES['movies_keywords'] = (
    "CREATE TABLE movies_keywords ("
      " movie_id int NOT NULL,"
      " keyword_id text NOT NULL"
      ") ENGINE=InnoDB")

TABLES['actors'] = (
    "CREATE TABLE actors ("
    " actor_id int NOT NULL AUTO_INCREMENT," 
    " actor_name text NOT NULL,"
    " PRIMARY KEY (actor_id)"
    ") ENGINE=InnoDB")

TABLES['movie_actors'] = (
    "CREATE TABLE movie_actors("
    " movie_id int NOT NULL,"
    " actor_id int NOT NULL"
    ") ENGINE=InnoDB")

create_table(TABLES, 'Mod_1_Project')

Creating table main_movie_table: OK
Creating table genres: already exists.
Creating table key_words: already exists.
Creating table movies_genres: already exists.
Creating table movies_keywords: already exists.
Creating table actors: already exists.
Creating table movie_actors: already exists.


In [ ]:
#insert data into tables
def add_movies_toDB():
    for movie in movie_dict:
        ('''insert into main_movie_table (title, release_date, award_wins, award_nomination, 
        RT_rating, IMDB_rating) values(%s,%s,%s,%s,%s,%s)''',
        movie['Title'], movie['Released'], movie['Awards'], 
        movie['Rating'][1]['Value'].replace('%', ''), movie['imdbRating']) 
        conn.commit
         
def add_genres_toDB():
    for movie in movie_dict:
        ('''insert into genres (genre) values(%s)''', movie['Genre'])
        conn.commit
            
## Drop all tables to refresh
def drop_everything():
    cursor.execute("""DROP TABLE Reviews;""")
    cnx.commit()
    cursor.execute("""DROP TABLE Businesses;""")
    cnx.commit()